In [1]:
# importing libraries
from datetime import datetime
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

import glob
%matplotlib inline
import pandas as pd

import os


In [2]:
from threading import Thread

class ThreadedCamera(object):
    def __init__(self, source='cam:Sayda@0.0.0.0:88/videoMain'):

        self.capture = cv2.VideoCapture(f'rtsp://{source}')

        self.thread = Thread(target = self.update, args = ())
        self.thread.daemon = True
        self.thread.start()

        self.status = False
        self.frame  = None

    def update(self):
        while True:
            if self.capture.isOpened(): (self.status, self.frame) = self.capture.read()
            else: continue

    def grab_frame(self):
        if self.status: return self.frame
        else: return None 
    
    def end_frame(self):
        self.capture.release()

In [3]:
def mouse_click(event, x, y, flags, param):
    global ROI_Region
    global img_original
    global saved_img
    global filename
    global counter
    
    if event == cv2.EVENT_RBUTTONDOWN:
        counter = counter +1
        print(counter)
        filename = str(counter)+'.jpg'
        print(filename)
        cv2.imwrite(filename, saved_img)
#         print("After saving image:")  
#         print(os.listdir(directory))
        print('Successfully saved')        
        cv2.destroyAllWindows()
    elif event == cv2.EVENT_MBUTTONDOWN: ROI_Region = [0,0,0,0]
    else: pass

In [4]:
streamer = ThreadedCamera('cam:Sayda@0.0.0.0:88/videoMain')


In [5]:
counter = 1
filename = ''

saved_img = cv2.imread('C:/Users/s.ezati/Desktop/verification/people/1.jpg')


In [6]:
directory = r'C:/Users/s.ezati/Desktop/database'
os.chdir(directory)

In [ ]:
while True: 
    img_original = streamer.grab_frame()
    
    try: img_original = Image.fromarray(img_original)
    except AttributeError: continue
       
    saved_img = cv2.resize(np.array(img_original), (1250, 750))

    
    
    cv2.imshow('database', np.array(saved_img))

    cv2.setMouseCallback('database', mouse_click)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        cv2.destroyAllWindows()
        streamer.end_frame()
        break